In [ ]:
#Cleaned Version
from sklearn.preprocessing import LabelEncoder

# Data Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/data/train_data.csv', delimiter=',')
train_results = pd.read_csv('/content/drive/MyDrive/data/train_results.csv', delimiter=',')

X = train_data['text'].copy()
y = train_results['target'].copy()

le = LabelEncoder()
le.fit(y.unique())
y = le.transform(y) #positive:2, negative:0, neutral:1
y[:5]


Mounted at /content/drive


array([2, 0, 2, 2, 0])

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import re
import string

from nltk.stem.porter import PorterStemmer

# Drop the two columns 
#df.drop(["text_wo_stopfreq", "text_wo_stopfreqrare"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


nltk.download('stopwords')
english_stopwords = stopwords.words("english")
other= ["..", "...", "....", "... ..."]
negative_words = ('no', 'not')
english_stopwords = [e for e in english_stopwords if e not in negative_words]

tweet_tokenizer = TweetTokenizer(
    preserve_case=False,
    strip_handles=True,
    reduce_len=False
)

tokens = []
# for tweet in x_train:
# gets rid of URLs
X = X.apply(lambda x: re.sub(r'http\S+', '', x))
X = X.apply(lambda x: remove_urls(x))
X = X.apply(lambda x: stem_words(x))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(tokens)

0

In [ ]:
for i, tweet in enumerate(X):
    #print(stem_wordstweet)
    #tweet = stem_words(tweet)
    #print(tweet)
    # brazilian
    # if 'traido' in tweet: 
    #   continue 
    # if 'îºî' in tweet: 
    #   continue
    # if 'cap des' in tweet: 
    #   continue
    # if "àª" in tweet: 
    #   continue
    
    string_encode = tweet.encode("ascii", "ignore")
    tweet = string_encode.decode()
    tweet = tweet.replace(r"tooooooo", "too")
    tweet = tweet.replace(";)", "smile")
    tweet = tweet.replace(":-D", "lovw")
    tweet = tweet.replace("=]", "love")
    tweet = tweet.replace("xxx", "kiss")
    tweet = tweet.replace("thx", "thanks")
    tweet = tweet.replace("hhhhhh", "")
    tweet = tweet.replace("ooooooooooooooooooooo", "")
    tweet = tweet.replace("f......g", "fucking")
    tweet = tweet.replace("aaaa", "a")

    # removes digits
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    token = tweet_tokenizer.tokenize(tweet)
    token_clened = [word for word in token if (word not in string.punctuation and word not in other)]
    #if i % 200 == 0:
    #  print(token_clened)
    #token_clened = [word for word in token if (word not in other and word not in string.punctuation)]
    #print(token_clened)
    tokens.append(token_clened)

# Pad the X Sequences to Size 32 

Convert the y_train to categroial 

In [ ]:
# tokenize the sentences
from keras.preprocessing.text import Tokenizer

from keras_preprocessing.sequence import pad_sequences
# fit on the 20000
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(tokens)
print(len(tokenizer.word_counts))

from keras.utils import to_categorical

y_train = to_categorical(y, 3)
print(len(tokens))
train_text_vec = tokenizer.texts_to_sequences(tokens)
train_text_vec = pad_sequences(train_text_vec, maxlen=32)

254585


In [ ]:
len(train_text_vec)

1040323

# BiLSTM Variations

I built several different biLSTM variations.

In [ ]:
# we might want to change these
MAX_SEQ_LEN = 32 #this is based on a quick analysis of the len of sequences train['text'].apply(lambda x : len(x.split(' '))).quantile(0.95)
DEFAULT_BATCH_SIZE = 512

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, classification_report, log_loss

from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Flatten
from keras.layers import Dropout, Conv1D, GlobalMaxPool1D, GRU, GlobalAvgPool1D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping


from tensorflow.keras.layers import (Embedding,
                                     LSTM,
                                     Dense,
                                     Dropout,
                                     GlobalMaxPool1D,
                                     BatchNormalization)
def train(model, 
          X_train, y_train, X_test, y_test, 
          checkpoint_path='model.hdf5', 
          epochs = 25, 
          batch_size = DEFAULT_BATCH_SIZE, 
          class_weights = None, 
          fit_verbose=2,
          print_summary = True
         ):
    m = model()
    if print_summary:
        print(m.summary())
    m.fit(
        X_train, 
        y_train, 
        #this is bad practice using test data for validation, in a real case would use a seperate validation set
        validation_data=(X_test, y_test),  
        epochs=epochs, 
        batch_size=batch_size,
        class_weight=class_weights,
         #saves the most accurate model, usually you would save the one with the lowest loss
        callbacks= [
            ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=False),
            EarlyStopping(patience = 2)
        ],
        verbose=1
    ) 
    print("\n\n****************************\n\n")
    print('Loading Best Model...')s
    return m #returns best performing model


def model_1d():
  VECTOR_FEATURES = 32
  lstm_model = Sequential()
  # could be smaller
  lstm_model.add(Embedding(len((tokenizer.word_counts))+1,
                      VECTOR_FEATURES,
                      input_length= MAX_SEQ_LEN))
  lstm_model.add(LSTM(280, return_sequences = True))
  lstm_model.add(GlobalMaxPool1D())
  lstm_model.add(BatchNormalization())
  lstm_model.add(Dropout(0.5))
  lstm_model.add(Dense(10, activation="relu"))
  lstm_model.add(Dropout(0.25))
  lstm_model.add(Dense(3, activation = "sigmoid"))
  lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return lstm_model


def model_1c():
    """
    Adding dropout to reduce overfitting using a bidiretional LSTM
    """
    model = Sequential()
    model.add(Embedding(input_dim = (len((tokenizer.word_counts))+1), output_dim = 108, input_length = 32))
    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(LSTM(108, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
    model.add(Conv1D(64, 4))
    model.add(GlobalMaxPool1D())
    model.add(Dense(108, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



# Bi LSTM, Larger Model

In [ ]:
y_train.shape

(1040323, 3)

In [ ]:
train_text_vec.shape

(1040323, 32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_text_vec, y_train, train_size = 0.7, random_state=42)

In [ ]:

print(len(tokenizer.word_counts))
model = train(model_1d, 
           X_train,
           y_train,
           X_val,
           y_val,
           checkpoint_path='/content/drive/MyDrive/Colab Notebooks/DataChallenge2/_model_1c.h5',
           print_summary = True
          )


254585
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 32)            8146752   
                                                                 
 lstm (LSTM)                 (None, 32, 280)           350560    
                                                                 
 global_max_pooling1d (Globa  (None, 280)              0         
 lMaxPooling1D)                                                  
                                                                 
 batch_normalization (BatchN  (None, 280)              1120      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 280)               0         
                                                                 
 dense (Dense)               (None, 10)          

Best Model (81.2%)


In [ ]:
model_1c = train(model_1c, 
           X_train,
           y_train,
           X_val,
           y_val,
           epochs=2,
           checkpoint_path='/content/drive/MyDrive/Colab Notebooks/DataChallenge2/_model_1d.h5',
           print_summary = True
          )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 108)           27495288  
                                                                 
 spatial_dropout1d (SpatialD  (None, 32, 108)          0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 32, 216)          187488    
 l)                                                              
                                                                 
 conv1d (Conv1D)             (None, 29, 64)            55360     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                        

# Running the Test Dataset

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/data/test_data.csv")
X_test = test_data['text'].copy()



tokens_test = []
X_test = X_test.apply(lambda x: re.sub(r'http\S+', '', x))
X_test = X_test.apply(lambda x: remove_urls(x))
for i, tweet in enumerate(X_test):

    string_encode = tweet.encode("ascii", "ignore")
    tweet = string_encode.decode()
    tweet = tweet.replace(r"tooooooo", "too")
    tweet = tweet.replace(";)", "smile")
    tweet = tweet.replace(":-D", "lovw")
    tweet = tweet.replace("=]", "love")
    tweet = tweet.replace("xxx", "kiss")
    tweet = tweet.replace("thx", "thanks")
    tweet = tweet.replace("hhhhhh", "")
    tweet = tweet.replace("ooooooooooooooooooooo", "")
    tweet = tweet.replace("f......g", "fucking")
    tweet = tweet.replace("aaaa", "a")
    tweet = tweet.replace("iii", "")
    tweet = tweet.replace("uhhhhhmmm", "um")
    tweet = tweet.replace("hahahahaah", "haha")
    tweet = tweet.replace("hahaha", "haha")
    tweet = tweet.replace("plllleeeaassseeeee", "please")
    tweet = tweet.replace("cccccrrrrrrrrrrrrryyyyyyyyy", "cry")
    tweet = tweet.replace("ssssssssss", "s")
    tweet = tweet.replace("grrrrrrr", "gr")
    tweet = tweet.replace(":]", "happy")
    tweet = tweet.replace("ll", "")
    tweet = tweet.replace("luvvv", "love")
    token = tweet_tokenizer.tokenize(tweet)
    

    token_clened = [word for word in token if (word not in string.punctuation and word not in other)]
    #print(token_clened)
    #if i % 200 == 0:
    #  print(token_clened)
    #token_clened = [word for word in token if (word not in other and word not in string.punctuation)]
    #print(token_clened)
    tokens_test.append(token_clened)

In [ ]:
# prepping evaluation data
import numpy as np

test_text_vec = tokenizer.texts_to_sequences(tokens_test)
test_text_vec = pad_sequences(test_text_vec, maxlen=32)

model_bi_lstm_predictions = model_1c.predict(test_text_vec)
predictions = np.argmax(model_bi_lstm_predictions, axis=1)


file = open("/content/drive/MyDrive/no_stop_words_processed_with_bltm_removal_of_other_parts.csv","a")
file.write("id,target\n")
count = 0 
for prediction in predictions:
    output = str(int(prediction))
    file.write(str(count) + "," + output + "\n")
    count = count + 1 
file.close()

17506/17506 [==============================] - 332s 19ms/step


# Conformer RNNT & Larger BLSTM

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
MAX_SEQ_LEN=32
inputs = layers.Input(shape=(MAX_SEQ_LEN,))
# to do: make this smaller?
embedding_layer = TokenAndPositionEmbedding(MAX_SEQ_LEN, len(tokenizer.word_counts)+1, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(108, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(3, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
# use much larger batch sizes 
# monitor only the eval set for the losses + stop after 5 {early stopping criteron}
# batch size, epcohs+ etc
#  X_train,
          #  y_train,
          #  X_val,
          #  y_val,


history = model.fit(
    X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), 
    callbacks= [
            ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/DataChallenge2/transformer.md", 
                            monitor='val_loss', 
                            verbose=1,
                            mode="auto",
                            save_best_only=True),
            EarlyStopping(patience = 2)
        ],
        verbose=1
)

In [ ]:
def model_1c_larger():
    """
    Adding dropout to reduce overfitting using a bidiretional LSTM
    """
    model = Sequential()
    model.add(Embedding(input_dim = (len((tokenizer.word_counts))+1), output_dim = 216, input_length = 32))
    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(LSTM(216, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
    model.add(Conv1D(64, 4))
    model.add(GlobalMaxPool1D())
    model.add(Dense(216, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model_1c_larger = train(model_1c_larger, 
           X_train,
           y_train,
           X_val,
           y_val,
           epochs=2,
           checkpoint_path='/content/drive/MyDrive/Colab Notebooks/DataChallenge2/_model_1d.h5',
           print_summary = True
          )

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 32, 216)           54990576  
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 32, 216)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional_1 (Bidirectio  (None, 32, 432)          748224    
 nal)                                                            
                                                                 
 conv1d_1 (Conv1D)           (None, 29, 64)            110656    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                      

In [ ]:
# Uses stemming + removes punctution

In [ ]:
X = train_data['text'].copy()
nltk.download('stopwords')
english_stopwords = stopwords.words("english")
other= ["..", "...", "....", "... ..."]
negative_words = ('no', 'not')
english_stopwords = [e for e in english_stopwords if e not in negative_words]

tweet_tokenizer = TweetTokenizer(
    preserve_case=False,
    strip_handles=True,
    reduce_len=False
)

tokens = []
# for tweet in x_train:
# gets rid of URLs
X = X.apply(lambda x: re.sub(r'http\S+', '', x))
X = X.apply(lambda x: remove_urls(x))
X = X.apply(lambda x: stem_words(x))



for i, tweet in enumerate(X):
    
    string_encode = tweet.encode("ascii", "ignore")
    tweet = string_encode.decode()
    tweet = tweet.replace(r"tooooooo", "too")
    tweet = tweet.replace(";)", "smile")
    tweet = tweet.replace(":-D", "lovw")
    tweet = tweet.replace("=]", "love")
    tweet = tweet.replace("xxx", "kiss")
    tweet = tweet.replace("thx", "thanks")
    tweet = tweet.replace("hhhhhh", "")
    tweet = tweet.replace("ooooooooooooooooooooo", "")
    tweet = tweet.replace("f......g", "fucking")
    tweet = tweet.replace("aaaa", "a")
    # stems the tweet 
    tweet = stem_words(tweet)


    # removes digits
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    token = tweet_tokenizer.tokenize(tweet)
    token_clened = [word for word in token if (word not in string.punctuation and word not in other)]
    if i % 1000 == 0:
      print(token_clened)
    #token_clened = [word for word in token if (word not in other and word not in string.punctuation)]
    #print(token_clened)
    tokens.append(token_clened)

In [ ]:
# tokenize the sentences
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences
# fit on the 20000
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(tokens)
print(len(tokenizer.word_counts))

from keras.utils import to_categorical
y_train = to_categorical(y, 3)
print(len(tokens))
train_text_vec = tokenizer.texts_to_sequences(tokens)
train_text_vec = pad_sequences(train_text_vec, maxlen=32)


224484
1040323


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_text_vec, y_train, train_size = 0.7, random_state=42)

In [ ]:
X_train.shape

(728226, 32)

In [ ]:
from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(train_text_vec, y_train, train_size = 0.7, random_state=42)
model_1c_larger = train(model_1c_larger, 
           X_train,
           y_train,
           X_val,
           y_val,
           epochs=2,
           checkpoint_path='/content/drive/MyDrive/Colab Notebooks/DataChallenge2/_model_1d.h5',
           print_summary = True
          )

In [ ]:
# prepping evaluation data
import numpy as np

test_text_vec = tokenizer.texts_to_sequences(tokens_test)
test_text_vec = pad_sequences(test_text_vec, maxlen=32)

model_bi_lstm_predictions = model_1c_larger.predict(test_text_vec)
predictions = np.argmax(model_bi_lstm_predictions, axis=1)


file = open("/content/drive/MyDrive/second_try_no_stop_words_processed_with_bltm_removal_of_other_parts.csv","a")
file.write("id,target\n")
count = 0 
for prediction in predictions:
    output = str(int(prediction))
    file.write(str(count) + "," + output + "\n")
    count = count + 1 
file.close()

17506/17506 [==============================] - 332s 19ms/step
